# Monte Carlo Simulation for FE
## IEOR 4703

### Calculting Greeks via simulation (Black-Merton-Scholes)

1. w/o common random numbers
2.  w/ common random numbers
3. via pathwise estimator
4. via likelihood ratio

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
# parameters
s0 = 1000
K = 1000
sig = 0.20
T = 1
r = 0.04
q = 0.0115

delS = 0.1
delSig = 0.01

nN = 500000

### Exact values for $\Delta$, $\Gamma$, and $vega$

In [3]:
d1 = (np.log(s0/K) + (r - q + sig**2/2)*T)/(sig*np.sqrt(T))
d2 = d1 - sig*np.sqrt(T)

delC =  np.exp(-q*T) * norm.cdf(d1)
# delP = -np.exp(-q*T)* norm.cdf(-d1)

gammaC = np.exp(-q*T)*norm.pdf(d1)/(s0*sig*np.sqrt(T))
# gammaP = gammaC

vegaC = s0*np.sqrt(T)*np.exp(-q*T)*norm.pdf(d1)/100
# vegaP = vegaC

In [4]:
np.random.seed(64814561)

In [5]:
z1 = np.random.randn(nN)
z2 = np.random.randn(nN)
z3 = np.random.randn(nN)

### $\Delta$ & $\Gamma$

In [6]:
tmp1U = (s0 + delS)*np.exp((r-q-sig**2/2)*T)
tmp1  =  s0      *np.exp((r-q-sig**2/2)*T)
tmp1D = (s0-delS)*np.exp((r-q-sig**2/2)*T)

tmp2 = sig*np.sqrt(T)

# ---------------------------------
# (1) W/ Common Random Numbers
s = tmp1*np.exp(tmp2*z1)
tmp = np.exp(-r*T)*np.maximum(s-K,0)

s = tmp1U*np.exp(tmp2*z1)
tmpU = np.exp(-r*T)*np.maximum(s-K,0)

s = tmp1D*np.exp(tmp2*z1)
tmpD = np.exp(-r*T)*np.maximum(s-K,0)

delC_CRN1 = np.mean((tmpU - tmp))/delS
delC_CRN2 = np.mean((tmpU - tmpD))/(2*delS)

gam_CRN  = np.mean((tmpU - 2*tmp + tmpD))/(delS**2)

# ----------------------------------
# (2) W/O Common Random Numbers
s =  tmp1*np.exp(tmp2*z1)
tmp = np.exp(-r*T)*np.maximum(s - K,0)

s = tmp1U*np.exp(tmp2*z2)
tmpU = np.exp(-r*T)*np.maximum(s - K,0)

s = tmp1D*np.exp(tmp2*z3)
tmpD = np.exp(-r*T)*np.maximum(s - K,0)

delC_tilde1 = np.mean((tmpU - tmp))/delS
delC_tilde2 = np.mean((tmpU - tmpD))/(2*delS)

gam_tilde  = np.mean((tmpU - 2*tmp + tmpD))/(delS**2)

# ---------------------------------
# (3) Pathwise Estimator
s =  tmp1*np.exp(tmp2*z1)
dhds0 = np.exp(-r*T)*(s>K)*s/s0
delC_pathwise = np.mean(dhds0)

gam_pathwise = np.nan

# ----------------------------------
# (4) Likelihood ratio
scoreH_delta = np.exp(-r*T)*np.maximum(s - K,0)*z1/(s0*sig*np.sqrt(T))
delC_likelihood = np.mean(scoreH_delta)

scoreH_gamma = (z1**2 - 1)/(s0**2*sig**2*T) - z1/(s0**2*sig*np.sqrt(T))
scoreH_gamma *= np.exp(-r*T)*np.maximum(s - K,0)
gam_likelihood = np.mean(scoreH_gamma)

### $Vega$

In [7]:
tmp1U  =  s0 * np.exp((r - q - (sig + delSig)**2/2)*T)
tmp1D  =  s0 * np.exp((r - q - (sig - delSig)**2/2)*T)

tmp2U = (sig + delSig) * np.sqrt(T)
tmp2D = (sig - delSig) * np.sqrt(T)

# (1) W/ Common Random Numbers
s = tmp1U * np.exp(tmp2U*z1)
tmpU = np.exp(-r*T) * np.maximum(s-K,0)

s = tmp1D * np.exp(tmp2D*z1)
tmpD = np.exp(-r*T) * np.maximum(s-K,0)

vega_CRN = np.mean((tmpU - tmpD)/(2*delSig))/100

# (2) W/O Common Random Numbers
s = tmp1U * np.exp(tmp2U*z1)
tmpU = np.exp(-r*T) * np.maximum(s-K,0)

s = tmp1D * np.exp(tmp2D*z2)
tmpD = np.exp(-r*T) * np.maximum(s-K,0)

vega_tilde = np.mean((tmpU - tmpD)/(2*delSig))/100

# (3) Pathwise Estimator
s =  tmp1 * np.exp(tmp2*z1)
dhdsig = np.exp(-r*T) * (s>K) * (s/sig) * (np.log(s/s0) - (r - q + sig**2/2)*T)
vega_pathwise = np.mean(dhdsig)/100

# (4) Likelihood Ratio

#scoreH = exp(-r*T)*np.maximum(s-K,0)*(sig*(sig*T-np.sqrt(T)*z1)-1)/sig
scoreH = np.exp(-r*T)*np.maximum(s - K,0)*((z1**2-1)/sig - z1*np.sqrt(T))
vega_likelihood = np.mean(scoreH)/100

### Displaying results

In [8]:
print('==================')
print('\tDelta')
print('==================')
print('Exact: %f' % delC)
print('W/ CRN: %f' % delC_CRN1)
print('W/O CRN: %f' % delC_tilde1)
print('Pathwise estimator: %f' % delC_pathwise)
print('Likelihood estimator: %f' % delC_likelihood)
print('==================')
print('\tGamma')
print('==================')
print('Exact: %f' % gammaC)
print('W/ CRN: %f' % gam_CRN)
print('W/O CRN: %f' % gam_tilde)
print('Pathwise estimator: %f' % gam_pathwise)
print('Likelihood estimator: %f' % gam_likelihood)
print('==================')
print('\tVega')
print('==================')
print('Exact: %f' % vegaC)
print('W/ CRN: %f' % vega_CRN)
print('W/O CRN: %f' % vega_tilde)
print('Pathwise estimator: %f' % vega_pathwise)
print('Likelihood estimator: %f' % vega_likelihood)

	Delta
Exact: 0.588991
W/ CRN: 0.589834
W/O CRN: 0.435417
Pathwise estimator: 0.589745
Likelihood estimator: 0.589117
	Gamma
Exact: 0.001915
W/ CRN: 0.001999
W/O CRN: -4.401623
Pathwise estimator: nan
Likelihood estimator: 0.001926
	Vega
Exact: 3.829535
W/ CRN: 3.827000
W/O CRN: 3.835041
Pathwise estimator: 3.826954
Likelihood estimator: 3.851186
